In [1]:
import os
import struct

file_name = "../../casia/1241-c.pot"
outfile = '../data/1241-c.npz'  # filename for .npz file

# Iterate files
data_file = open(file_name, 'rb')
total_bytes = os.path.getsize(file_name)
print("total_bytes = ", total_bytes)
current_bytes = 0
word_count = 0

# Iterate characters
char_list = []
while current_bytes != total_bytes:
    data_length, = struct.unpack('H', data_file.read(2))  # 这一组数据字节数
    # print("data_length = ", data_length)
    
    tag_code, = struct.unpack('I', data_file.read(4))
    # print("tag_code = ", tag_code, tag_code.to_bytes(2, byteorder="big").decode("GBK"))
    
    stroke_number, = struct.unpack('H', data_file.read(2))  # 这一组数据有多少stroke
    # print("stroke_number = ", stroke_number)

    current_bytes += data_length
    all_x_coor = []  # 里面有很多组stroke的坐标
    all_y_coor = []
    # Iterate strokes
    for stroke_index in range(stroke_number):
        stroke_x_coor = []  # 每一组stroke的坐标序列
        stroke_y_coor = []
        # Iterate coordinates of points
        while 1:
            x_buff = data_file.read(2)
            y_buff = data_file.read(2)
            x, = struct.unpack('h', x_buff)
            y, = struct.unpack('h', y_buff)
            if x == -1:
                break
            stroke_x_coor.append(x)
            stroke_y_coor.append(y) 
        if len(stroke_x_coor) > 1:
            all_x_coor.append(stroke_x_coor)
            all_y_coor.append(stroke_y_coor)
    end_tag1, = struct.unpack('h', data_file.read(2))
    end_tag2, = struct.unpack('h', data_file.read(2))

    char_list.append((all_x_coor, all_y_coor))
    # break
    
print("char num = ", len(char_list))

total_bytes =  1068100
char num =  3755


In [2]:
print(len(char_list))
print(char_list[0][0])
print(char_list[0][1])

for i in range(len(char_list)):
    cnt = 0
    x_sum = 0
    y_sum = 0
    for j in range(len(char_list[i][0])):
        for k in range(len(char_list[i][0][j])):
            x_sum += char_list[i][0][j][k]
            cnt += 1
            y_sum += char_list[i][1][j][k]
    # print(x_sum, y_sum, cnt)
    x_avg = x_sum / cnt
    y_avg = y_sum / cnt
    for j in range(len(char_list[i][0])):
        for k in range(len(char_list[i][0][j])):
            char_list[i][0][j][k] -= x_avg
            char_list[i][1][j][k] -= y_avg
        """
        for k in range(len(char_list[i][0][j]), 0 , -1)
            char_list[i][0][j][k] -= char_list[i][0][j][k-1]
            char_list[i][1][j][k] -= char_list[i][1][j][k-1]
        """


3755
[[476, 476, 495, 495], [496, 498, 511, 524, 528, 506, 504, 504, 510, 540, 539], [570, 569, 576, 589, 575, 593, 591, 588, 580, 573, 564, 558, 555], [559, 559, 558, 553, 544, 541], [629, 664, 673, 674, 668], [615, 614, 613, 619, 626, 630, 634, 649, 631, 621, 616, 615, 633, 640, 645, 645], [660, 659, 655, 650, 640, 631, 619, 605, 584]]
[[6779, 6781, 6828, 6825], [6790, 6785, 6774, 6773, 6779, 6833, 6835, 6835, 6830, 6804, 6793], [6734, 6734, 6730, 6736, 6784, 6809, 6813, 6818, 6819, 6816, 6808, 6794, 6778], [6755, 6756, 6824, 6854, 6914, 6915], [6746, 6739, 6733, 6730, 6729], [6783, 6791, 6806, 6813, 6801, 6791, 6789, 6788, 6816, 6833, 6839, 6840, 6831, 6825, 6815, 6801], [6760, 6766, 6834, 6900, 6899, 6891, 6876, 6853, 6825]]


In [3]:
from matplotlib import pyplot as plt
import numpy as np
from rdp import rdp

def smooth(i, eps):
    x = char_list[i][0]
    y = char_list[i][1]

    strokes = []
    for i in range(len(x)):
        cur_stroke = []
        for j in range(len(x[i])):
            cur_stroke.append([x[i][j], y[i][j]])
        strokes.append(cur_stroke)

    new_strokes = []
    for i in range(len(strokes)):
        new_strokes.append(rdp(strokes[i], epsilon=eps))

    new_x = []
    new_y = []
    for i in range(len(strokes)):
        cur_x = []
        cur_y = []
        for j in range(len(new_strokes[i])):
            cur_x.append(new_strokes[i][j][0])
            cur_y.append(new_strokes[i][j][1])
        new_x.append(cur_x)
        new_y.append(cur_y)
    
    points = []
    for i in range(len(new_x)):
        for j in range(len(new_x[i])):
            if j != len(new_x[i]) - 1:
                points.append([new_x[i][j], new_y[i][j], 0])
            else:
                points.append([new_x[i][j], new_y[i][j], 1])
                
    for i in range(len(points) - 1, 0, -1):
        points[i][0] -= points[i-1][0]
        points[i][1] -= points[i-1][1]
    points[0][0] = 0
    points[0][1] = 0
    
    return np.array(points)

def draw(x, y, new_x, new_y):
    for i in range(len(x)):
        plt.plot(x[i], y[i], color='black', linewidth=3)
    plt.show()
    for i in range(len(x)):
        plt.plot(new_x[i], new_y[i], color='black', linewidth=3)
    plt.show()

In [4]:
def draw(delta_gt_stroke, delta_stroke):
    # (16, 3), (16, 3)
    ground_truth_stroke = delta_gt_stroke.copy()
    stroke = delta_stroke.copy()
    
    # convert to absolute coordinate
    scale_factor = 300
    low_tri_matrix = np.tril(np.ones((delta_gt_stroke.shape[0], delta_gt_stroke.shape[0])), 0)
    ground_truth_stroke[:, :2] = np.rint(scale_factor * np.matmul(low_tri_matrix, delta_gt_stroke[:, :2]))
    low_tri_matrix = np.tril(np.ones((delta_stroke.shape[0], delta_stroke.shape[0])), 0)
    stroke[:, :2] = np.rint(scale_factor * np.matmul(low_tri_matrix, delta_stroke[:, :2]))

    plt.figure(figsize=(6, 3))
    plt.subplot(121)
    # plt.xlim(0, 300)
    # plt.ylim(0, 300)
    pre_i = 0
    for i in range(ground_truth_stroke.shape[0]):
        if ground_truth_stroke[i][2] == 1:
            plt.plot(ground_truth_stroke[pre_i:i + 1, 0], ground_truth_stroke[pre_i:i + 1, 1], color='black', linewidth=3)
            pre_i = i + 1
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.title("Groud Truth")

    plt.subplot(122)
    # plt.xlim(0, 300)
    # plt.ylim(0, 300)
    pre_i = 0
    cnt = 0
    tot = 0
    for s in stroke:
        if s[2] == 1:
            tot += 1
    print("tot = ", tot)
    for i in range(stroke.shape[0]):
        if stroke[i][2] == 1:
            cnt += 1
            if cnt < int(tot / 2):
                plt.plot(stroke[pre_i:i + 1, 0], stroke[pre_i:i + 1, 1], color='black', linewidth=3)  # 9 / 13
            else:
                plt.plot(stroke[pre_i:i + 1, 0], stroke[pre_i:i + 1, 1], color='black', linewidth=3)  # 9 / 13
            pre_i = i + 1
    print("cnt = ", cnt)
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.title("Generated")

    plt.show()

In [5]:
dataset = []
for i in range(len(char_list)):
    points = smooth(i, 8)
    dataset.append(points)

np.savez(outfile, test_data=dataset)


In [33]:
data = np.load('../data/FZTLJW_775.npz', allow_pickle=True, encoding='latin1')
data.files

['train', 'std_train', 'test', 'std_test', 'valid', 'std_valid']

In [39]:
file1 = np.load('../data/FZTLJW_775.npz', allow_pickle=True, encoding='latin1')
file2 = np.load('../data/FZTLJW_775.npz', allow_pickle=True, encoding='latin1')

draw(file1['test_data'][0], file2['test_data'][0])